In [51]:
!pip install nltk

import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [52]:
movie_data = load_files('./data/sentiment_dataset')
X, y = movie_data.data, movie_data.target

In [53]:
print(f'length of X: {len(X)}')
print(f'length of X: {len(y)}')

print(X[0])
print(y[0])

length of X: 1386
length of X: 1386
b'cast : casper van dien , jane march , steven waddingham director : carl schenkel producers : stanley canter , dieter geissler , michael lake screenplay : baynard johnson and j . anderson black based on the " tarzan " stories by edgar rice burroughs cinematography : paul gilpin music : christopher franke u . s . distributor : warner brothers this is the kind of movie that makes one appreciate disney\'s live- action george of the jungle . tarzan and the lost city , the latest attempt to bring edgar rice burrough\'s legendary hero to the big screen , is one of the most inept and ill-timed of any tarzan adventure so far . badly conceived and poorly executed , tarzan and the lost city appears headed for a quick trip to video store shelves . i have no idea why the producers chose now to bring back tarzan ; it\'s not as if there are legions of new fans clamoring for his next movie . furthermore , it\'s even more curious that this film is rated pg , which 

In [54]:
# remove special characters

documents = []

from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

In [55]:
#from sklearn.feature_extraction.text import CountVectorizer
#vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
#X = vectorizer.fit_transform(documents).toarray()

# TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = tfidfconverter.fit_transform(documents).toarray()

In [56]:
# training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [57]:
print(f'length of X: {len(X_train)}')
print(f'length of X: {len(X_test)}')

length of X: 1108
length of X: 278


In [59]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [60]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[113  18]
 [ 41 106]]
              precision    recall  f1-score   support

           0       0.73      0.86      0.79       131
           1       0.85      0.72      0.78       147

    accuracy                           0.79       278
   macro avg       0.79      0.79      0.79       278
weighted avg       0.80      0.79      0.79       278

0.7877697841726619
